In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        img = np.dstack((band_1, band_2, band_3))
        images.append(std_img(img))

    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
def ConvBlock(model, layers, filters):
    '''Create [layers] layers consisting of zero padding, a convolution with [filters] 3x3 filters and batch normalization. Perform max pooling after the last layer.'''
    for i in range(layers):
        model.add(ZeroPadding2D((1, 1)))
        model.add(Conv2D(filters, (3, 3), activation='relu'))
        model.add(BatchNormalization(axis=3))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()

    # Input image: 75x75x3
    model.add(Lambda(lambda x: x, input_shape=(75, 75, 3)))
    ConvBlock(model, 1, 32)
    # 37x37x32
    ConvBlock(model, 1, 64)
    # 18x18x64
    ConvBlock(model, 1, 128)
    # 9x9x128
    ConvBlock(model, 1, 128)
    # 4x4x128
    model.add(Flatten())
    model.add(Dense(1,activation='sigmoid'))
    
    return model
print('model model')


model model


In [5]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator

def lr_f(epoch):
    if epoch<10:
        return 0.001
    elif epoch<30:
        return 0.0005
    else:
        return 0.0001

def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            #zoom_range=0.05,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 32
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=50, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

Epoch 1/50
Epoch 00001: val_loss improved from inf to 0.53231, saving model to best_m.h5
 - 3s - loss: 0.7306 - acc: 0.6784 - val_loss: 0.5323 - val_acc: 0.7731
Epoch 2/50
Epoch 00002: val_loss did not improve
 - 2s - loss: 0.4694 - acc: 0.7677 - val_loss: 0.6203 - val_acc: 0.7357
Epoch 3/50
Epoch 00003: val_loss did not improve
 - 2s - loss: 0.4479 - acc: 0.7910 - val_loss: 0.5538 - val_acc: 0.7531
Epoch 4/50
Epoch 00004: val_loss improved from 0.53231 to 0.53094, saving model to best_m.h5
 - 2s - loss: 0.4043 - acc: 0.8017 - val_loss: 0.5309 - val_acc: 0.7805
Epoch 5/50
Epoch 00005: val_loss did not improve
 - 2s - loss: 0.4468 - acc: 0.8060 - val_loss: 0.7589 - val_acc: 0.7157
Epoch 6/50
Epoch 00006: val_loss did not improve
 - 2s - loss: 0.3858 - acc: 0.8212 - val_loss: 0.6695 - val_acc: 0.7556
Epoch 7/50
Epoch 00007: val_loss did not improve
 - 2s - loss: 0.3471 - acc: 0.8375 - val_loss: 0.6194 - val_acc: 0.7531
Epoch 8/50
Epoch 00008: val_loss did not improve
 - 2s - loss: 0.2883

Epoch 14/50
Epoch 00014: val_loss did not improve
 - 2s - loss: 0.2507 - acc: 0.8860 - val_loss: 0.7424 - val_acc: 0.7406
Epoch 15/50
Epoch 00015: val_loss improved from 0.31662 to 0.29978, saving model to best_m.h5
 - 2s - loss: 0.2424 - acc: 0.8882 - val_loss: 0.2998 - val_acc: 0.8579
Epoch 16/50
Epoch 00016: val_loss improved from 0.29978 to 0.29282, saving model to best_m.h5
 - 2s - loss: 0.2288 - acc: 0.8992 - val_loss: 0.2928 - val_acc: 0.8828
Epoch 17/50
Epoch 00017: val_loss did not improve
 - 2s - loss: 0.2395 - acc: 0.8992 - val_loss: 0.4780 - val_acc: 0.8354
Epoch 18/50
Epoch 00018: val_loss did not improve
 - 2s - loss: 0.2568 - acc: 0.8843 - val_loss: 0.4865 - val_acc: 0.8180
Epoch 19/50
Epoch 00019: val_loss improved from 0.29282 to 0.28005, saving model to best_m.h5
 - 2s - loss: 0.2381 - acc: 0.8947 - val_loss: 0.2800 - val_acc: 0.8853
Epoch 20/50
Epoch 00020: val_loss did not improve
 - 2s - loss: 0.2183 - acc: 0.9045 - val_loss: 0.6219 - val_acc: 0.7706
Epoch 21/50
Ep

Epoch 25/50
Epoch 00025: val_loss did not improve
 - 2s - loss: 0.2134 - acc: 0.9096 - val_loss: 0.2284 - val_acc: 0.9027
Epoch 26/50
Epoch 00026: val_loss did not improve
 - 2s - loss: 0.2819 - acc: 0.8871 - val_loss: 0.3622 - val_acc: 0.8603
Epoch 27/50
Epoch 00027: val_loss did not improve
 - 2s - loss: 0.2204 - acc: 0.9104 - val_loss: 0.2512 - val_acc: 0.9052
Epoch 28/50
Epoch 00028: val_loss improved from 0.21613 to 0.20710, saving model to best_m.h5
 - 2s - loss: 0.2383 - acc: 0.9003 - val_loss: 0.2071 - val_acc: 0.9102
Epoch 29/50
Epoch 00029: val_loss did not improve
 - 2s - loss: 0.2351 - acc: 0.9079 - val_loss: 0.2792 - val_acc: 0.8828
Epoch 30/50
Epoch 00030: val_loss did not improve
 - 2s - loss: 0.2192 - acc: 0.9009 - val_loss: 0.4507 - val_acc: 0.8180
Epoch 31/50
Epoch 00031: val_loss did not improve
 - 2s - loss: 0.2007 - acc: 0.9119 - val_loss: 0.2160 - val_acc: 0.8978
Epoch 32/50
Epoch 00032: val_loss did not improve
 - 2s - loss: 0.2008 - acc: 0.9138 - val_loss: 0.241

Epoch 00038: val_loss did not improve
 - 2s - loss: 0.1497 - acc: 0.9324 - val_loss: 0.2889 - val_acc: 0.9002
Epoch 39/50
Epoch 00039: val_loss did not improve
 - 2s - loss: 0.1620 - acc: 0.9358 - val_loss: 0.2984 - val_acc: 0.9002
Epoch 40/50
Epoch 00040: val_loss did not improve
 - 2s - loss: 0.1521 - acc: 0.9428 - val_loss: 0.3120 - val_acc: 0.8678
Epoch 41/50
Epoch 00041: val_loss did not improve
 - 2s - loss: 0.1667 - acc: 0.9265 - val_loss: 0.3311 - val_acc: 0.8778
Epoch 42/50
Epoch 00042: val_loss did not improve
 - 2s - loss: 0.1564 - acc: 0.9355 - val_loss: 0.2806 - val_acc: 0.9102
Epoch 43/50
Epoch 00043: val_loss did not improve
 - 2s - loss: 0.1635 - acc: 0.9276 - val_loss: 0.3452 - val_acc: 0.8878
Epoch 44/50
Epoch 00044: val_loss did not improve
 - 2s - loss: 0.1475 - acc: 0.9389 - val_loss: 0.2985 - val_acc: 0.9002
Epoch 45/50
Epoch 00045: val_loss did not improve
 - 2s - loss: 0.1595 - acc: 0.9400 - val_loss: 0.3707 - val_acc: 0.8579
Epoch 46/50
Epoch 00046: val_loss di

In [6]:
import pickle
with open('../features/cnn_4_aug1_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_4_aug1_sub.csv', index=False)

0.219121964073
         id  is_iceberg
0  5941774d    0.012779
1  4023181e    0.935562
2  b20200e4    0.004924
3  e7f018bb    0.997598
4  4371c8c3    0.005455
